In [ ]:
#Importing and Installing Libraries
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re
!pip install transformers
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import random
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Setting up the GPU device
if torch.cuda.is_available():      
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [ ]:
#Importing Data
train = pd.read_csv('/content/drive/My Drive/ML-Project/train.csv')
val = pd.read_csv('/content/drive/My Drive/ML-Project/validation.csv')
test = pd.read_csv('/content/drive/My Drive/ML-Project/test.csv')
print(train.columns)

Index(['id', 'tweet', 'label'], dtype='object')


In [ ]:
#Encoding Label for "fake" and "real"
encoder = preprocessing.LabelEncoder()
train['label'] = encoder.fit_transform(train['label'])
val['label'] = encoder.transform(val['label'])

In [ ]:
#Removing Links and mentions from tweets
def removeLinks(text):
  text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
  text = re.sub(r'@\S+', '', text)
  return text

train['tweet'] = train['tweet'].apply(lambda x: removeLinks(x))
val['tweet'] = val['tweet'].apply(lambda x: removeLinks(x))
test['tweet'] = test['tweet'].apply(lambda x: removeLinks(x))

In [ ]:
MAX_LENGTH = 512

#Tokenizing using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize(sent):
  #Encode the text into token ids and Add '[CLS]' and '[SEP]'
  encoded = tokenizer.encode(sent, add_special_tokens = True, truncation=True, max_length = MAX_LENGTH)
  return encoded

train['encoded_tokens'] = train['tweet'].apply(lambda x: tokenize(x.lower()))
val['encoded_tokens'] = val['tweet'].apply(lambda x: tokenize(x.lower()))
test['encoded_tokens'] = test['tweet'].apply(lambda x: tokenize(x.lower()))

In [ ]:
#Padding Data
train['encoded_tokens'] = pad_sequences(train.encoded_tokens, maxlen=MAX_LENGTH, padding='post', value=0).tolist()
val['encoded_tokens'] = pad_sequences(val.encoded_tokens, maxlen=MAX_LENGTH, padding='post', value=0).tolist()
test['encoded_tokens'] = pad_sequences(test.encoded_tokens, maxlen=MAX_LENGTH, padding='post', value=0).tolist()

In [ ]:
#Masking to identify tokens that are words i.e. id>0
def masking(tokens):
  mask = [int(token > 0) for token in tokens]
  return mask
train['masks'] = train['encoded_tokens'].apply(lambda x: masking(x))
val['masks'] = val['encoded_tokens'].apply(lambda x: masking(x))
test['masks'] = test['encoded_tokens'].apply(lambda x: masking(x))

In [ ]:
#creating Tensors for the data
train_inputs = torch.tensor(train.encoded_tokens)
validation_inputs = torch.tensor(val.encoded_tokens)
train_labels = torch.tensor(train.label)
validation_labels = torch.tensor(val.label)
train_masks = torch.tensor(train.masks)
validation_masks = torch.tensor(val.masks)

In [ ]:
#Calculating Accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
#SETTING THE PARAMETERS
BATCH_SIZE = 8
LR = 2e-5
EPOCH = 16
EPS = 1e-8

In [ ]:
#Creating Data Loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

#Defining the Model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = False
)

#Set the running environment as the Device
model.cuda()

#Setting Learning Rate and Eps value
optimizer = AdamW(model.parameters(), lr = LR, eps = EPS)

total_steps = len(train_dataloader) * EPOCH
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                        num_warmup_steps = 0, 
                                        num_training_steps = total_steps)
#Traing the Model
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
      
#Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, EPOCH):
  #Clearing the Cache
  torch.cuda.empty_cache()

  print("Epoch: {:}".format(epoch_i+1))
  total_loss = 0
  model.train()
  
  # ========================================
  #                Training
  # ========================================
  for step, batch in enumerate(train_dataloader):
    #unpacking the batch
    b_input_ids = batch[0].to(device) #input
    b_input_mask = batch[1].to(device)  #masks
    b_labels = batch[2].to(device)  #labels
    
    model.zero_grad()    
    outputs = model(b_input_ids, 
                  token_type_ids=None, 
                  attention_mask=b_input_mask, 
                  labels=b_labels)
    loss = outputs[0]
    total_loss += loss.item()
    #Loss Propogates backward for updating values
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  avg_train_loss = total_loss / len(train_dataloader)  
  loss_values.append(avg_train_loss)
  
  # ========================================
  #               Validation
  # ========================================
  model.eval()
  #Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  #Evaluate data for one epoch
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():  
      outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
        
      #Calculate the accuracy for this batch of test sentences.
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
      #Accumulate the total accuracy.
      eval_accuracy += tmp_eval_accuracy
      #Track the number of batches
      nb_eval_steps += 1
  #Report the final accuracy for this validation run.
  print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 1
Accuracy: 0.97
Epoch: 2
Accuracy: 0.96
Epoch: 3
Accuracy: 0.97
Epoch: 4
Accuracy: 0.97
Epoch: 5
Accuracy: 0.97
Epoch: 6
Accuracy: 0.96
Epoch: 7
Accuracy: 0.97
Epoch: 8
Accuracy: 0.97
Epoch: 9
Accuracy: 0.98
Epoch: 10
Accuracy: 0.97
Epoch: 11
Accuracy: 0.97
Epoch: 12
Accuracy: 0.97
Epoch: 13
Accuracy: 0.98
Epoch: 14
Accuracy: 0.98
Epoch: 15
Accuracy: 0.98
Epoch: 16
Accuracy: 0.98


In [ ]:
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

In [ ]:
#Creating Prediction Data Loaders
prediction_inputs = torch.tensor(test.encoded_tokens)
prediction_masks = torch.tensor(test.masks)
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)

#Saving the validation Predictions
model.eval()
predictions = []
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  #Unpacking
  b_input_ids, b_input_mask = batch
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  label_ids = np.argmax(logits, axis=1).flatten()
  predictions.extend(label_ids.tolist())

In [ ]:
sol = pd.DataFrame(test['id'], columns=['id'])
sol['label'] = encoder.inverse_transform(predictions)
sol.to_csv('/content/drive/My Drive/ML-Project/bert8_test.txt', index=False)